In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

In [2]:
#ich habe als unabhängige Variable folgende
# - Temperatur
# - Preis
# - Uhrzeit - mit Sin und Cos ausgedrückt um das zyklische Verhalten zu bekommen. 
#   Ich habe weiters auch noch sin(2Pi*x/12) und mit cos hinzugefügt um Vor- und Nachmittagsspitzen darzustelle. Ich bin mir ned sicher ob dies nötig/richtig ist
# - Lag: ich habe eine Abhängigkeit von der vorherigen Last hinzugefügt, ich weiß wieder nicht ob das gewollt ist, es verbessert den R² aber deutlich. 

df_price = pd.read_csv("preise2023.csv")
df_load = pd.read_csv("load2023.csv")
hours = pd.Series(range(8760))    #csv eingelesen und hours erstellt
df_temp = pd.read_csv("MessstationenStundendaten.csv")
timeseries = list(range(24))*365
clock = pd.Series(timeseries)
df_sinclock = np.sin(2 * np.pi * clock / 24)
df_cosclock = np.cos(2 * np.pi * clock / 24)
df_sinclock_half = np.sin(2 * np.pi * clock / 12)
df_cosclock_half = np.cos(2 * np.pi * clock / 12)

df_temp.rename(columns={'tl': 'Temp'}, inplace=True)
df_price.rename(columns={'AT': 'Price'}, inplace=True)
df_load.rename(columns={'load': 'Load'}, inplace=True)  #csv Spaltennamen renamed
df_combined = pd.concat([hours, clock, df_price['Price'], df_load['Load'], df_temp['Temp']], axis=1) #daten in ein dataframegepackt 
df_combined.rename(columns={df_combined.columns[0]: 'hours'}, inplace=True) #hours spalte in hours umbenannt
df_combined.rename(columns={df_combined.columns[1]: 'clock'}, inplace=True) #hours spalte in clock umbenannt
df_combined['Load_lagged'] = df_combined['Load'].shift(1) #Setzt meinen Wert in Zusammenhang mit dem t-1 Wert
#df_combined['Sin_clock'] = df_sinclock                  #Uhrzeit durch Sin ausdrücken ist nicht signifikant
df_combined['Cos_clock'] = df_cosclock                  #Uhrzeit durch Cos ausdrücken
df_combined['Sin_clock_half'] = df_sinclock_half               #Uhrzeit durch Sin ausdrücken
df_combined['Cos_clock_half'] = df_cosclock_half                  #Uhrzeit durch Cos ausdrücken
df_combined = df_combined[df_combined['Price'] > 0]     #Preise unter 0 weggecuttet
df_combined = df_combined.fillna(df_combined.mean())    #Tauscht Nan-Werte mit Spaltenmittelwert aus (Temp. hatte Nan-Werte)


In [3]:
X = df_combined[['Price', 'Temp','Cos_clock','Sin_clock_half','Cos_clock_half','Load_lagged']]  # Unabhängige Variablen definieren
X = sm.add_constant(X)  # Konstanten-Term hinzufügen (Intercept)

y = df_combined['Load']  # Zielvariable

model = sm.OLS(y, X).fit()  # OLS-Regression (Ordinary Least Squares)

# Ergebnisse ausgeben
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Load   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                 3.325e+04
Date:                Thu, 03 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:10:17   Log-Likelihood:                -60371.
No. Observations:                8623   AIC:                         1.208e+05
Df Residuals:                    8616   BIC:                         1.208e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            674.4551     25.469     26.

In [55]:

# Test ob Variablen Signifikant sind mit ChatJPT - nicht genauer betrachtet

# 1. Erstelle univariate Modelle und teste die Signifikanz jeder Variable
significant_vars = []

# Setze das Signifikanzniveau (alpha) auf 5%
alpha = 0.05

for column in X.columns:
    # Univariates Modell
    X_temp = sm.add_constant(X[column])  # Füge einen konstanten Term hinzu
    model = sm.OLS(y, X_temp).fit()
    
    # p-Wert der Variable
    p_value = model.pvalues[0]  # p-Wert der einzigen unabhängigen Variable
    
    # Teste, ob der p-Wert kleiner als 0.2 ist (Signifikanzniveau)
    if p_value <= alpha:
        significant_vars.append(column)  # Wenn signifikant, behalte die Variable

# 2. Zeige die signifikanten Variablen
print(f"Signifikante Variablen (p-Wert <= 0.2): {significant_vars}")


Signifikante Variablen (p-Wert <= 0.2): ['const', 'Price', 'Temp', 'Cos_clock', 'Sin_clock_half', 'Cos_clock_half', 'Load_lagged']
